# Geometric Distortions due to Gradient Inhomogeneities

The software we are developing will allows us to measure geometric distortion from MR images of a grid phantom.

Geometric distortion originates from inhomogeneities in the magnetic field.  In particular

- from a non-uniform static field $B_0(\vec{r})$
- from non-linear gradient magnets $\vec{G}(\vec{r})$
- from susceptibility changes caused by the sample

We are working on understanding how non-linearities in the gradient magnets manifests as geometric distortion.

A key question that we have been considering is:

**Is the geometric distortion originating from gradient magnet inhomogeneities different during slice-selection vs within the slice (i.e. from scanning the 2D k-space within the slice)?**

This seems like an important question because if the geometric distortion *is* different along the slice direction from within the slice, then the geometric distortion in general would depend upon the slice-selection direction.  For example, the geometric distortion could be different for a set of axial slices vs sagittal slices.

This would mean that our software would need to operate on a single set of slices, and would not be able to combine multiple sets of slices from different directions.

We note that, in practice, the distortion may in fact be similar due to how the gradient magnets are constructed.

## Current Understanding

Our current understanding is that, yes, the geometric distortion caused by gradient magnet inhomogeneities is different along the slice-selection direction from within the slice.

What follows is our explanation of why we believe this.

When performing "multi-slice 2D imaging", a slice within the 3D volume is selected by varying the slice-selection gradient and the central frequency and bandwidth of the initial $pi/2$ RF pulse.

The RF pulse excites the "iso-surface" wherein the magnetic field strength leads to matching Larmour frequencies.  Ideally the "iso-surface" is a plane perpendicular to the slice-selection gradient, however gradient non-linearities would warp the plane, leading to geometric distortion.

After a slice is selected, the scanner still needs to scan through the 2D k-space to localize the contrast within the selected slice.  The gradient magnets are used to scan through this k-space.

It seems that gradient magnet inhomogeneities present during k-space scanning would to manifest differently than during slice selection, because they would cause artifacts in the frequency domain.

These frequency domain artifacts can be seen from the 1D MRI frequency imaging equation (equation 9.14 of *Magnetic Resonance Imaging: Physical Principles and Sequence Design, 2nd edition*):


$s(t) = \int dz \; \rho(z)e^{i\phi_G(z, t)}$

In this equation $s(t)$ is the measured signal on the receive coil, and $\rho(z)$ is the effective spin density.

The phase term, $\phi_G(z, t)$, results from the time-and-space varying gradient field (the phase from the static field has already been canceled out).  In particular, we have:

$\phi_G(z, t) = - \int_0^t   dt'\; \omega_G(z, t') = - \int_0^t    dt'\gamma G(z, t')$

In the case that $G(z, t) = zG(t)$, we end up with a Fourier transform relating $s$ and $\rho$

$s(k) = \int dz \rho(z) e^{-i2\pi k z}$

where

$k(t) = \gamma' \int_0^t dt' G(t')$.

These last two equations are 9.15 and 9.16 in the MRI text.

If, on the other hand, $G(z, t)$ is not linear, we end up with something more complicated.

For example, if we have $G(z, t) = (z + a_2 z^2 + a_3 z^3) G(t)$, we no longer have a Fourier transform, but we have:

$s(k) = \int dz \; \rho(z)e^{- i 2 \pi (z + a_2 z^2 + a_3 z^3) k}$

Presumably, despite these extra terms, the measured signal $s(k)$ will still be inverse-Fourier transformed as usual.

It is not exactly clear how the non-linearity will manifest itself, however it does appear to be different than how gradient non-linearities manifest during slice selection.

## Plotting Simple Case

We can visualize this result using a simple simulation.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

z = np.linspace(-10, 10, 2000)
dz = np.diff(z[:2])
a_2 = 0.01
a_3 = 0.004

plt.plot(z, z + a_2*z**2 + a_3*z**3)
plt.title('A Non-linear Gradient Field')
plt.ylabel('Gradient Field Strength [a.u.]')
plt.xlabel('Position [a.u.]')

In [ ]:
rho_actual = np.zeros(z.shape, dtype=complex)
rho_actual[(-8 < z) & (z < -5)] = 1.0
rho_actual[(1 < z) & (z < 4)] = 1.8

plt.plot(z, np.abs(rho_actual))
plt.title('A Simple, Fabricated Spin Density')
plt.ylabel('Effective Spin Density [a.u.]')
plt.xlabel('Position [a.u.]')
plt.ylim([0, 2])


In [ ]:

def simulate_1d_mri(rho_actual, gradient):
    '''    
    NOTE: this doesn't use the FFT because I am lazy and it is small.
    
    s(k) = \int dz \; \rho(z)e^{- i 2 \pi (z + a_2 z^2 + a_3 z^3) k} 
    '''
    k = np.linspace(-20, 20, 4000)
    dk = np.diff(k[:2])
    s = np.sum(rho_actual*np.exp(-2j*np.pi*np.outer(k, gradient)), axis=1)*dz
    rho_measured = np.sum(s*np.exp(2j*np.pi*np.outer(z, k)), axis=1)*dk
    return np.abs(rho_measured)

plt.plot(z, simulate_1d_mri(rho_actual, z + a_2*z**2 + a_3*z**3), 'r', label='Measured Spin Density (w distortion)')
plt.plot(z, simulate_1d_mri(rho_actual, z), 'g', label='Measured Spin Density (no distortion)')
plt.plot(z, np.abs(rho_actual), 'b', label='Actual Spin Density')
plt.title('A Simple, Fabricated Spin Density')
plt.ylabel('Effective Spin Density [a.u.]')
plt.xlabel('Position [a.u.]')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

Note that we see Gibb's phenomena, which is expected because we are cutting off high frequency components of the sharp edge in $rho$.

Even with this simulation, it is not completely clear what the answer to the question is.

Qualitatively, it seems like they are different, but it is difficult to say for sure.  For example, the distortions along the slice-axis should not alter the amplitude, however the non-linear gradient does appear to alter the amplitude within the slice.

It is also possible that the values for $a_2$ and $a_3$ are unrealistically high, and that this may affect the practical conclusion.